## Predicción de la clasificacion final de varios países utilizando un modelo basado en el sistema de puntuación ELO

Imports

In [1]:
import pandas as pd
from google.colab import files
import numpy as np
import scipy.optimize as optim
import matplotlib.pyplot as plt

Funciones

In [2]:
#función nA
def my_logisticA(xa, ha, da):
  return (3 / (1 + 10**(-(xa+ha)/da)))

#función nB
def my_logisticB(xb, hb, db):
  return (3 / (1 + 10**((xb+hb)/db)))

#Devuelve el índice en la lista del nombre del equipo introducido
def getindexteam(team, classification):
  i = 0
  for t in classification:
    if t[0] == team:
      return i
    else:
      i = i+1

In [3]:
#Función para procesamiento de partidos y predicción
def elomodel(file,partidosjugados):

  #Lectura del archivo de partidos
  #!rm file
  #archive = files.upload()
  data = pd.read_csv(file, sep=";",  encoding='latin1')

  classification = []
  nA = []
  nB = []

  #Recoge el nombre de los equipos
  for i in range(10):
    classification.append([data.loc[i][0], 0, 0])
    classification.append([data.loc[i][1], 0, 0])

  #Recoge los puntos obtenidos según el marcador del partido y los valores de diferencia pA-pB y puntos obtenidos para la función sigmoide que se calculará
  for i in range(partidosjugados):
    tupleA = []
    tupleB = []
    A = data.loc[i][0]
    B = data.loc[i][1]
    pA = classification[getindexteam(A,classification)][1]
    pB = classification[getindexteam(B,classification)][1]
    tupleA.append(pA-pB)
    tupleB.append(pA-pB)
    if data.loc[i][2] > data.loc[i][3]:
      indexteam = getindexteam(data.loc[i][0],classification)
      classification[indexteam][1] += 3
      tupleA.append(3)
      tupleB.append(0)
    elif data.loc[i][2] == data.loc[i][3]:
      indexteam = getindexteam(data.loc[i][0],classification)
      classification[indexteam][1] += 1
      indexteam = getindexteam(data.loc[i][1],classification)
      classification[indexteam][1] += 1
      tupleA.append(1)
      tupleB.append(1)
    else: 
      indexteam = getindexteam(data.loc[i][1],classification)
      classification[indexteam][1] += 3
      tupleA.append(0)
      tupleB.append(3)
    nA.append(tupleA)
    nB.append(tupleB)
    

  dfa = pd.DataFrame(nA, columns =['x', 'y'])
  dfb = pd.DataFrame(nB, columns =['x', 'y'])

  #convierte las columnas del dataframe en arrays
  xa = np.array(dfa['x']) 
  ya = np.array(dfa['y'])
  xb = np.array(dfb['x']) 
  yb = np.array(dfb['y'])


  #Optimización de los parámetros para obtener los adecuados
  (ha,da),cova = optim.curve_fit(my_logisticA, xa, ya)
  print("Valores de h y d respectivamente para la funcion nA")
  print(ha,da)
  print("\n")

  (hb,db),covb = optim.curve_fit(my_logisticB, xb, yb)
  print("Valores de h y d respectivamente para la funcion nB")
  print(hb,db)
  print("\n")

  #Aplicación de las funciones a los partidos restantes que se deben predecir
  for i in range(partidosjugados,380):
    A = data.loc[i][0]
    B = data.loc[i][1]
    pA = classification[getindexteam(A,classification)][1]
    pB = classification[getindexteam(B,classification)][1]
    valorA = my_logisticA(pA-pB,ha,da)
    valorB = my_logisticB(pA-pB,hb,db)
    classification[getindexteam(A,classification)][2] += valorA 
    classification[getindexteam(B,classification)][2] += valorB 

  #Suma los puntos obtenidos de partidos jugados a los puntos predecidos
  for x in classification:
    x[2] = x[2] + x[1]

  #Ordena e imprime la clasificación
  classification = sorted(classification,key=lambda x: x[2], reverse = True)
  print("Clasificación obtenida")
  print(classification)
  print("\n")

  #pinta la función
  fig, axs = plt.subplots(1,2, figsize=(15,5))

  dfordered = dfa.sort_values('x')
  xo = np.array(dfordered['x']) 
  yo = np.array(dfordered['y'])
  axs[0].set_title("Función nA")
  axs[0].set_ylabel('Número de puntos ganados por el equipo A')
  axs[0].set_xlabel('Diferencia de puntos entre equipo A y B')
  axs[0].scatter(xa, ya, c="red")
  axs[0].plot(xo,(3 / (1 + 10**(-(xo+ha)/da))))
  print("\n")

  dfordered = dfb.sort_values('x')
  xo = np.array(dfordered['x']) 
  yo = np.array(dfordered['y'])
  axs[1].set_title("Función nB")
  axs[1].set_ylabel('Número de puntos ganados por el equipo B')
  axs[1].set_xlabel('Diferencia de puntos entre equipo A y B')
  axs[1].scatter(xb, yb, c="red")
  axs[1].plot(xo,(3 / (1 + 10**((xo+hb)/db))))
  print("\n")

  return classification

#Función que devuelve el error respecto a la clasificación obtenida
def errorcalculation(guyonclassification, classification):

  #Cálculo del error entre los valores obtenidos y los resultados del modelo de Guyon
  #Imprime el error de cada equipo y la media del error de todos los equipos
  pointdifference = []
  for i in range(20):
    pointdifference.append(-(guyonclassification[i][2] - classification[i][2]))

  mediapuntosdesviados = 0
  for t in pointdifference:
    mediapuntosdesviados += abs(t)

  mediapuntosdesviados=mediapuntosdesviados/20
    
  print("Error respecto al resultado del modelo de Guyon para cada equipo")
  print(pointdifference)
  print("\n")
  print("Media del error total "+  str(mediapuntosdesviados))
  print("\n")

Predicción Ligue 1 19-20

In [ ]:
classification = elomodel("Ligue1Data1920.csv", 279)
guyonclassificationfrance = [['Paris S-G', 68, 94.58], ['Marseille', 56, 75.56], ['Rennes', 50, 68.49], ['Lille', 49, 67.64], ['Reims', 41, 54.95], ['Montpellier', 40, 54.65], ['Nice', 41, 54.53], ['Lyon', 40, 54.47], ['Monaco', 40, 54.19], ['Angers', 39, 52.87], ['Strasbourg', 38, 52.18], ['Nantes', 37, 51.81], ['Bordeaux', 37, 51.39], ['Metz', 34, 45.26], ['Brest', 34, 44.48], ['Saint-Étienne', 30, 40.73], ['Dijon', 30, 40.29], ['Nimes', 27, 36.65], ['Amiens', 23, 31.17], ['Toulouse', 13, 18.83]]
errorcalculation(guyonclassificationfrance, classification)

Predicción Premier League 19-20

In [ ]:
classification = elomodel("PremierData1920.csv", 288)
guyonclassificationengland = [['Liverpool', 82, 105.9], ['Manchester City', 57, 77.8], ['Leicester City', 53, 70.8], ['Chelsea', 48, 62.9], ['Manchester Utd', 45, 60.4], ['Wolves', 43, 57.4], ['Sheffield Utd', 43, 57.3], ['Tottenham', 41, 54.2], ['Arsenal', 40, 53.0], ['Burnley', 39, 51.1], ['Crystal Palace', 39, 49.6], ['Everton', 37, 48.4], ['Newcastle Utd', 35, 46.3], ['Southampton', 34, 45.1], ['Brighton', 29, 36.5], ['West Ham', 27, 36.4], ['Watford', 27, 35.4], ['Bournemouth', 27, 34.3], ['Aston Villa', 25, 32.6], ['Norwich City', 21, 28.3]]
errorcalculation(guyonclassificationengland, classification)

Predicción La Liga 19-20

In [ ]:
classification = elomodel("LaLigaData1920.csv", 270)